In [0]:
%pip install yfinance numpy pandas scikit-learn tensorflow

In [0]:
import yfinance as yf

df = yf.download('TSLA', start='2010-06-29', progress=False)
print("Columns in downloaded DataFrame:", df.columns.tolist())
print(df.head())

In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt

# Title
print("Tesla Stock Price Prediction (LSTM)")

model = tf.keras.models.load_model('/Workspace/Users/rsangramofficial@gmail.com/EDA/Tesla Stock/lstm_tesla_model.h5', compile=False)
model.compile(optimizer='adam', loss='mse')

# Read Tesla stock data from local CSV file
csv_path = '/Workspace/Users/rsangramofficial@gmail.com/EDA/Tesla Stock/TSLA.csv'
df = pd.read_csv(csv_path)

# Use only 'Adj Close' for prediction
if 'Adj Close' not in df.columns:
    raise ValueError("'Adj Close' column not found in TSLA.csv. Available columns: " + str(df.columns.tolist()))
data = df[['Adj Close']]
scaler = MinMaxScaler()
scaler.fit(data)

seq_length = 60
scaled_data = scaler.transform(data)
last_seq = scaled_data[-seq_length:]

days = 5  # Fixed value for prediction

def predict_future(model, last_sequence, days):
    predictions = []
    current = last_sequence.copy()
    for _ in range(days):
        pred = model.predict(current.reshape(1, seq_length, 1), verbose=0)
        predictions.append(pred[0, 0])
        current = np.append(current[1:], pred)
    return scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

# Always run prediction
preds = predict_future(model, last_seq, days)
future_dates = pd.date_range(start=pd.to_datetime(df['Date'].iloc[-1]) + pd.Timedelta(days=1), periods=days, freq='B')
pred_df = pd.DataFrame({'Predicted': preds}, index=future_dates)

# Plot predictions vs actual
plt.figure(figsize=(16, 6))
plt.plot(pd.to_datetime(df['Date']).tail(100), data['Adj Close'].tail(100), label='Actual')
plt.plot(pred_df.index, pred_df['Predicted'], label='Predicted')
plt.title('Tesla Stock Price Prediction (LSTM)')
plt.legend()
plt.show()

# Print predictions
for i, p in enumerate(preds, 1):
    print(f"Day {i} ({future_dates[i-1].date()}): ${p:.2f}")